<a href="https://colab.research.google.com/github/moabe75/solvation_free_energy/blob/main/structure_preparation_RDKit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install the RDKit package and mols2grid molecule viewer
!pip install rdkit-pypi
!pip install mols2grid

In [2]:
# Import libraries
from rdkit import Chem
from rdkit.Chem import AllChem, Draw, Descriptors, PandasTools
from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.ipython_use_SVG = True
import mols2grid

import pandas as pd
import numpy as np
import os

In [4]:
# Get some info about the sdf file
sdf_file = "molecules_ChEMBL.sdf"
mols = PandasTools.LoadSDF(sdf_file)
mols.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2586 entries, 0 to 2585
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   chembl_id         2586 non-null   object
 1   chembl_pref_name  2586 non-null   object
 2   ID                2586 non-null   object
 3   ROMol             2586 non-null   object
dtypes: object(4)
memory usage: 101.0+ KB


In [ ]:
# Convert sdf files to SMILES
sdf_file = Chem.SDMolSupplier('molecules_ChEMBL.sdf')
smiles_file = open('smiles.smi', "w")
for mol in sdf_file:
  if mol is not None:
    smi = Chem.MolToSmiles(mol)
    smiles_file.write(f"{smi}\n")

In [6]:
# Total number of molecules after conversion
print('Total Number of Molecules =', len(open('smiles.smi', 'r').readlines()))

Total Number of Molecules = 2530


In [7]:
# Obtain info about the aromatic and aliphatic rings and molecular weight 
smiles_file = open('smiles.smi', 'r')
j = k = l = 0
for mol in smiles_file:
  mols = Chem.MolFromSmiles(mol) 
  if Descriptors.NumAromaticRings(mols) == 1: j += 1   
  if Descriptors.NumAromaticRings(mols) == 1 and Descriptors.NumAliphaticRings(mols) == 1: k += 1
  if Descriptors.ExactMolWt(mols) <= 150: l += 1
      
print('Total number of molecules with 1 aromatic ring =', str(j))
print('Total number of molecules with 1 aromatic ring and 1 aliphatic ring =', str(k), '\n')
print('Total number of molecules with molecular weight <= 150 =', str(l))

Total number of molecules with 1 aromatic ring = 2530
Total number of molecules with 1 aromatic ring and 1 aliphatic ring = 321 

Total number of molecules with molecular weight <= 150 = 2530


In [ ]:
# Let's visualize the molecular structures
smiles_file = open('smiles.smi', 'r')
mols = [Chem.MolFromSmiles(smi) for smi in smiles_file]
Draw.MolsToGridImage(mols, molsPerRow = 4, useSVG = True)
mols2grid.display(mols)

In [ ]:
# Add H atoms, optimize with the UFF force field, and convert to the XYZ coordinate 
smiles_file = open('smiles.smi', 'r')
k = 1
dirName = 'xyz_files'
for mol in smiles_file:
  mol = Chem.MolFromSmiles(mol)
  mol = Chem.AddHs(mol)
  AllChem.Compute2DCoords(mol)
  AllChem.EmbedMolecule(mol)
  AllChem.UFFOptimizeMolecule(mol)

  xyz_file = open(os.path.join(dirName,'mol_' + str(k) + '.xyz'), 'w')
  xyz_file.write(Chem.MolToXYZBlock(mol) + '\n')
  k += 1

In [ ]:
# Download all XYZ files for the DFT calculations
from google.colab import files
!zip -r /content/xyz_files.zip /content/xyz_files
files.download("/content/xyz_files.zip")